In [12]:
# Librarys
%run ForeSignalScraper.ipynb function(Login)
%run ForeSignalScraper.ipynb function(CustomizeFoSignals)
%run TradingViewScaper.ipynb function(Price)
%run OpenOrdersMT5.ipynb function(Login_mt5,Expiration_time,Determination_Set_Order)
%run Sql-Server.ipynb function(send_datas)
from datetime import datetime
from collections import OrderedDict
import pandas as pd
import random
import time

Rows deleted successfully.


In [13]:
# Currencys list and Desired Time to Refresh wbsites
Currencys_list = ['EURUSD','USDCHF','GBPUSD','USDJPY','USDCAD','AUDUSD','EURJPY','NZDUSD','GBPCHF']
desired_time = int(input('Please Enter Your Desired Time Range To load Websiter - input a number ex:20 : '))

# Foresignal login Information
FoLogin_url = 'https://foresignal.com/en/login/index'
Fouser_name = 'xabiz'
FoUser_password = 'tohid0010011'

# Foresignal Scrape Specification
FoWebsite_url = 'https://foresignal.com/en/'
Foclassorid = 'class'
Foclassoridvalue = 'd-flex flex-row flex-wrap signal-row'

#Login to Foresignal and Scrape Data
ForeSignalDriver = Login(FoLogin_url,Fouser_name,FoUser_password)

#Login to Tradingview and Scrape Data
final_list = Price(2,Currencys_list)

# Alpari Broker Demo Account
# login_id = 51915381
# password = 'dq5tttot'
# server = 'Alpari=MT5-Demo'

# FIBOGroup Broker Demo Account
# login_id = 3060009
# password = '$9Bj#5qL'
# server = 'FIBOGroup-MT5 Server'

#MetaTrader5 Login Informations and login to Software
login_id = 51915381
password = 'dq5tttot'
server = 'Alpari-MT5-Demo'
Login_mt5 (login_id,password,server)

# Empty Dicts and Lists
final_informations = OrderedDict()
Order_Dict = OrderedDict()
History = OrderedDict()
History_counter = 1
FoFinalList = list()
Reference_list = list()

# Use this line to create connection with sql server database
server = 'DESKTOP-7PVA1DT'
database = 'FInancial Scraper'
username = 'sa'
password = 'Mohsen13781378###***'
conn_str = connect_to_database(server,database,username,password)

Please Enter Your Desired Time Range To load Websiter - input a number ex:20 : 60


In [14]:

while True:

    FoFinalList_Start_len = len(FoFinalList)
    
    for items in CustomizeFoSignals(ForeSignalDriver,FoWebsite_url,Foclassorid,Foclassoridvalue):
        if len(items) > 10 and items not in FoFinalList:
            FoFinalList.append(items)
#     print(FoFinalList)
    
    FoFinalList_Final_len = len(FoFinalList)

    if FoFinalList_Start_len < FoFinalList_Final_len:
        
        Tradingview_price_dictionary = OrderedDict()

        for item in range (0,9):
            now = datetime.now()
            final_list[0].switch_to.window(final_list[0].window_handles[item])
            current_date = f'{Currencys_list[item]} : {now}'
            current_price = final_list[1][item].text
            
            Tradingview_price_dictionary[current_date] = current_price
            
#         print(Tradingview_price_dictionary)
        
        for item_fo in FoFinalList:
            for k,v in Tradingview_price_dictionary.items():
                if item_fo[0] in k[:6]:
                    item_fo.append(k)
                    item_fo.append(v)
                    final_informations[f'{item_fo[:11]}'] = item_fo
        
        Reference_list_start_len = len(Reference_list)

        Reference_list = list(final_informations.items()) 
        
        Reference_list_final_len = len(Reference_list)

        if Reference_list_start_len < Reference_list_final_len:
            
            for items in Reference_list[Reference_list_start_len:Reference_list_final_len]:
                
                Order_Dict[f'{items[1]}'] = items[1]
                
            for key, val in list(Order_Dict.items())[Reference_list_start_len:Reference_list_final_len]:
                        
                current_time = str(datetime.now()).split()[1].replace(':',' ').split()
            
                Order_hour = int(val[2].replace(':',' ').split()[0])
            
                Order_minute = int(val[2].replace(':',' ').split()[1])
            
                if int(current_time[0]) >= Order_hour :
            
#                     print(val)
                    Symbole = val[0]
                    Entry_Price = float(val[6])
                    Stoploss_Price = float(val[10])
                    Takeprofit_Price = float(val[8])
                    Current_Price = float(val[12])
                    current_time = val[4].replace(':',' ').split()
                    expiration = Expiration_time (int(current_time[0]),int(current_time[1]))
                                       
                    if val[5] == 'Buy at' and Entry_Price > Current_Price:
                        Type_of_Trade = 'buy stop'
                
                    elif val[5] == 'Buy at' and Entry_Price < Current_Price:
                        Type_of_Trade = 'buy limit'
            
                    elif val[5] == 'Sell at' and Entry_Price < Current_Price:
                        Type_of_Trade = 'sell stop'
            
                    elif val[5] == 'Sell at' and Entry_Price > Current_Price:
                        Type_of_Trade = 'sell limit'
                
                    request = Determination_Set_Order(Type_of_Trade, Symbole , 0.05 ,Entry_Price
                                              ,Stoploss_Price,Takeprofit_Price,expiration)   
                    result = mt.order_send(request)
                    result
                                        
                    TableName = 'Informations'
                    InfoTuple = (History_counter, val[0], val[1], val[2], val[3], val[4],val[5]
                                 , float(val[6]), float(val[8]), float(val[10]), val[11][8:], float(val[12]))
                                 
                    History_counter += 1
                    
                    send_datas(conn_str,TableName,InfoTuple)                    
                
                    for key in list(Order_Dict.keys()):
                        val = Order_Dict[key]
                        key_to_remove = key

                else :
                    continue
            
    delay_time = random.randrange(0,desired_time)
    time.sleep(delay_time)


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=119.0.6045.200)
Stacktrace:
	GetHandleVerifier [0x00007FF66CDF82B2+55298]
	(No symbol) [0x00007FF66CD65E02]
	(No symbol) [0x00007FF66CC205AB]
	(No symbol) [0x00007FF66CC00038]
	(No symbol) [0x00007FF66CC86BC7]
	(No symbol) [0x00007FF66CC9A15F]
	(No symbol) [0x00007FF66CC81E83]
	(No symbol) [0x00007FF66CC5670A]
	(No symbol) [0x00007FF66CC57964]
	GetHandleVerifier [0x00007FF66D170AAB+3694587]
	GetHandleVerifier [0x00007FF66D1C728E+4048862]
	GetHandleVerifier [0x00007FF66D1BF173+4015811]
	GetHandleVerifier [0x00007FF66CE947D6+695590]
	(No symbol) [0x00007FF66CD70CE8]
	(No symbol) [0x00007FF66CD6CF34]
	(No symbol) [0x00007FF66CD6D062]
	(No symbol) [0x00007FF66CD5D3A3]
	BaseThreadInitThunk [0x00007FF999EC7344+20]
	RtlUserThreadStart [0x00007FF99B2226B1+33]
